JN FOR FILLING REPORT GROSS PROFIT TABLE FOR UAM MYLEVO-GROSS PROFIT REPORT 

In [15]:
# CREATE DATABASE CONNECTION VIA ORATOR

from orator import DatabaseManager
from orator import Model

config = {
    'default': 'mysql',
    'mysql': {
        'driver': 'mysql',
        'host': 'localhost',
        'database': 'atc',
        'user': 'root',
        'password': 'password'
       
    }
}

db = DatabaseManager(config)
invoice = db.table('invoices')
consignment = db.table('consignments')
consignment_band = db.table('consignment_band')

##DAILY SALES

from datetime import date, datetime, timedelta

day_of_week = datetime.now().strftime("%w")
year = str(datetime.now().year)
month = str(datetime.now().month)
today = str(date.today())

def dailySales():
    weekday = today
    if day_of_week in ['1', '2', '3', '4', '5']:
        if db.table('invoices').where('net', '>', 0).where('route_code', 'S').where('invoice_date', '=', today).sum('net') is None:
            daily_sales = 0
            return daily_sales
        else:
            daily_sales = round(db.table('invoices').where('net', '>', 0).where('route_code', 'S').where('invoice_date', '=', today).sum('net'))
            return daily_sales
    elif day_of_week == '6':
        daily_sales = round(db.table('invoices').where('net', '>', 0).where('route_code', 'S').where('invoice_date', '=', (datetime.now() - timedelta(1)).strftime('%Y-%m-%d')).sum('net'))
        return daily_sales
    elif day_of_week == '0':
        daily_sales = round(db.table('invoices').where('net', '>', 0).where('route_code', 'S').where('invoice_date', '=', (datetime.now() - timedelta(2)).strftime('%Y-%m-%d')).sum('net'))
        return daily_sales
    
##DAILY GROSS PROFIT

def dailyGrossProft():
    day_of_week = datetime.now().strftime("%w")
    year = str(datetime.now().year)
    month = str(datetime.now().month)
    today = str(date.today())
    
    none = None
    daily_gp = []
    
    invoices = db.table('invoices').where('net', '>', 0).where('route_code', 'S').where('invoice_date', today).get()
    consignments = db.table('consignments').get()
    bands = db.table('consignment_band').get()
   
    #consignments = consignments.with_('invoices','bands').get()
    
    for consignment in consignments:
        for invoice in invoices:
            #calculate fixed rate
            if consignment.consignment_code == invoice.consignment_code and consignment.fixed == 1:
                gp = consignment.current_rate * invoice.net
                #print(gp)
                daily_gp.append(gp)
                
            elif consignment.consignment_code == invoice.consignment_code and consignment.fixed == 0:
                total = consignment.current_net + invoice.net
                #print(total)
                for band in bands:
                    if consignment.id == band.consignment_id:
                        #calculate at band level1
                        if band.low == 0 and total < band.high:
                            gp1 = band.percent * invoice.net
                            #print(gp1)
                            daily_gp.append(gp1)
                            #calculate at split level
                            
                        elif band.low == 0 and consignment.current_net < band.high and total > band.high:
                            curPortion = band.high - consignment.current_net
                            gp2 = band.percent * curPortion
                            #print(gp2)
                            daily_gp.append(gp2)
                            nextPortion = invoice.net - curPortion
                            next_band = db.table('consignment_band').where('consignment_id', band.consignment_id).where('low', band.high + 1).first()
                            gp3 = nextPortion * next_band.percent
                            #print(gp3)
                            break
                            
                            #calculate at band level2
                        elif band.low != 0 and band.high != none and band.low <= total <= band.high:
                            gp4 = band.percent * invoice.net
                            #print(gp4)
                            daily_gp.append(gp4)
                            
                        elif band.low != 0 and band.high != none and band.low <= consignment.current_net <= band.high and total > band.high:
                            curPortion = band.high - consignment.current_net
                            gp5 = band.percent * curPortion
                            #print(gp5)
                            total_yearly_gp.append(gp5)
                            nextPortion = invoice.net - curPortion
                            next_band = db.table('consignment_band').where('consignment_id', band.consignment_id).where('low', band.high + 1).first()
                            gp6 = nextPortion * next_band.percent
                            #print(gp6)
                            total_yearly_gp.append(gp6)
                            #dict[consignment.consignment_code] = total
                            break 

                            #calculate at band level3
                        elif consignment.current_net >= band.low and band.high == none:
                            gp7 = band.percent * invoice.net
                            #print(gp7)
                            daily_gp.append(gp7)
    
    return round(sum(daily_gp))

##DAILY GROSS PROFIT PERCENTAGE

def dailyGrossProfitPercentage():
    if dailySales() == 0:
        GPPercent = 0
    else:
        GPPercent = round((dailyGrossProft()/dailySales()*100),2)
    return GPPercent

##MONTHLY SALES

def monthlySales():
    year = str(datetime.now().year)
    month = str(datetime.now().month)
    today = str(date.today())
    monthly_sales = round(db.table('invoices').where('net', '>', 0).where('route_code', 'S').where('invoice_date', '<=', today).where('invoice_date', '>=', year + '-' + month + '-1').sum('net'))
    #monthly_sales = round(invoice.where('net', '>', 0).where('route_code', 'S').where('invoice_date', '<=', '2020-3-4').where('invoice_date', '>=', '2020-3-1').sum('net'))
    return monthly_sales

##MONTHLY GROSS PROFIT

def monthlyGrossProfit():
    day_of_week = datetime.now().strftime("%w")
    year = str(datetime.now().year)
    month = str(datetime.now().month)
    today = str(date.today())
 
    none = None
    daily_gp = []
    
    invoices = db.table('invoices').where('net', '>', 0).where('route_code', 'S').where('invoice_date', '<=', today).where('invoice_date', '>=', year + '-' + month + '-1').get()
    consignments = db.table('consignments').get()
    bands = db.table('consignment_band').get()
   
    #consignments = consignments.with_('invoices','bands').get()
    
    for consignment in consignments:
        for invoice in invoices:
            #calculate fixed rate
            if consignment.consignment_code == invoice.consignment_code and consignment.fixed == 1:
                gp = consignment.current_rate * invoice.net
                #print(gp)
                daily_gp.append(gp)
                
            elif consignment.consignment_code == invoice.consignment_code and consignment.fixed == 0:
                total = consignment.current_net + invoice.net
                #print(total)
                for band in bands:
                    if consignment.id == band.consignment_id:
                        #calculate at band level1
                        if band.low == 0 and total < band.high:
                            gp1 = band.percent * invoice.net
                            #print(gp1)
                            daily_gp.append(gp1)
                            #calculate at split level
                            
                        elif band.low == 0 and consignment.current_net < band.high and total > band.high:
                            curPortion = band.high - consignment.current_net
                            gp2 = band.percent * curPortion
                            print(gp2)
                            daily_gp.append(gp2)
                            nextPortion = invoice.net - curPortion
                            next_band = db.table('consignment_band').where('consignment_id', band.consignment_id).where('low', band.high + 1).first()
                            gp3 = nextPortion * next_band.percent
                            #print(gp3)
                            daily_gp.append(gp3)
                            break
                            
                            #calculate at band level2
                        elif band.low != 0 and band.high != none and band.low <= total <= band.high:
                            gp4 = band.percent * invoice.net
                            #print(gp4)
                            daily_gp.append(gp4)
                            
                        elif band.low != 0 and band.high != none and band.low <= consignment.current_net <= band.high and total > band.high:
                            curPortion = band.high - consignment.current_net
                            gp5 = band.percent * curPortion
                            #print(gp5)
                            daily_gp.append(gp5)
                            nextPortion = invoice.net - curPortion
                            next_band = db.table('consignment_band').where('consignment_id', band.consignment_id).where('low', band.high + 1).first()
                            gp6 = nextPortion * next_band.percent
                            #print(gp6)
                            daily_gp.append(gp6)
                            #dict[consignment.consignment_code] = total
                            break 

                            #calculate at band level3
                        elif consignment.current_net >= band.low and band.high == none:
                            gp7 = band.percent * invoice.net
                            #print(gp7)
                            daily_gp.append(gp7)
   
    return round(sum(daily_gp))
            
    
##MONTHLY GROSS PROFIT PERCENTAGE

def monthlyGrossProfitPercentage():
    GPPercent = round((monthlyGrossProfit()/monthlySales()*100),2)
    return GPPercent

##YEARLY SALES

def yearlySales():
    year = str(datetime.now().year)
    month = str(datetime.now().month)
    today = str(date.today())
    yearly_sales = round(db.table('invoices').where('net', '>', 0).where('route_code','S').where('invoice_date', '<=', today).where('invoice_date', '>=', year + '-' + '01-01').sum('net'))
    #yearly_sales = round(invoice.where('net', '>', 0).where('route_code', 'S').where('invoice_date', '<=', '2020-3-4').where('invoice_date', '>=', '2020-1-1').sum('net'))
    return yearly_sales

##YEARLY GROSS PROFIT

def yearlyGrossProfit():
    day_of_week = datetime.now().strftime("%w")
    year = str(datetime.now().year)
    month = str(datetime.now().month)
    today = str(date.today())

    none = None
    daily_gp = []

    invoices = db.table('invoices').where('net', '>', 0).where('route_code', 'S').where('invoice_date','<=', today).where('invoice_date','>=',year + '-01-01').get()
    consignments = db.table('consignments').get()
    bands = db.table('consignment_band').get()
   
    #consignments = consignments.with_('invoices','bands').get()

    for consignment in consignments:
        for invoice in invoices:
            #calculate fixed rate
            if consignment.consignment_code == invoice.consignment_code and consignment.fixed == 1:
                gp = consignment.current_rate * invoice.net
                #print(gp)
                daily_gp.append(gp)
                
            elif consignment.consignment_code == invoice.consignment_code and consignment.fixed == 0:
                total = consignment.current_net + invoice.net
                #print(total)
                for band in bands:
                    if consignment.id == band.consignment_id:
                        #calculate at band level1
                        if band.low == 0 and total < band.high:
                            gp1 = band.percent * invoice.net
                            #print(gp1)
                            daily_gp.append(gp1)
                        
                        elif band.low == 0 and consignment.current_net < band.high and total > band.high:
                            curPortion = band.high - consignment.current_net
                            gp2 = band.percent * curPortion
                            #print(gp2)
                            daily_gp.append(gp2)
                            nextPortion = invoice.net - curPortion
                            next_band = db.table('consignment_band').where('consignment_id', band.consignment_id).where('low', band.high + 1).first()
                            gp3 = nextPortion * next_band.percent
                            #print(gp3)
                            daily_gp.append(gp3)
                            break
                        
                          #calculate at band level2  
                        elif band.low != 0 and band.high != none and band.low <= total <= band.high:
                            gp4 = band.percent * invoice.net
                            #print(gp4)
                            daily_gp.append(gp4)
                            
                        elif band.low != 0 and band.high != none and band.low <= consignment.current_net <= band.high and total > band.high:
                            curPortion = band.high - consignment.current_net
                            gp5 = band.percent * curPortion
                            #print(gp5)
                            daily_gp.append(gp5)
                            nextPortion = invoice.net - curPortion
                            next_band = db.table('consignment_band').where('consignment_id', band.consignment_id).where('low', band.high + 1).first()
                            gp6 = nextPortion * next_band.percent
                            daily_gp.append(gp6)
                            #dict[consignment.consignment_code] = total
                            break

                        #calculate at band level2  
                        elif consignment.current_net >= band.low and band.high == none:
                            gp7 = band.percent * invoice.net
                            #print(gp7)
                            daily_gp.append(gp7)       
    
    return round(sum(daily_gp))

##YEARLY GROSS PROFIT PERCENTAGE

def yearlyGrossProfitPercentage():
    GPPercent = round((yearlyGrossProfit()/yearlySales()*100),2)
    return GPPercent

##DAILY MEMPHIS OWNED SALES

def dailyMemOwnedSales():
    day_of_week = datetime.now().strftime("%w")
    year = str(datetime.now().year)
    month = str(datetime.now().month)
    today = str(date.today())
    weekday = today
    
    invoices = db.table('invoices').where('net', '>', 0).where('route_code', 'S').where('invoice_date', today).get()
    consignments = db.table('consignments').where('uam_owned',1).where('active',1).get()
    
    total = []
    
    for consignment in consignments:
        for invoice in invoices:
            if consignment.consignment_code == invoice.consignment_code:
                total.append(invoice.net)
    sales = round(sum(total))
    return sales

##DAILY MEMPHIS OWNED GROSS PROFIT

def memOwnedDailyGP():
    day_of_week = datetime.now().strftime("%w")
    year = str(datetime.now().year)
    month = str(datetime.now().month)
    today = str(date.today())

    none = None
    daily_gp = []
    
    invoices = db.table('invoices').where('net', '>', 0).where('route_code', 'S').where('invoice_date', today).get()
    consignments = db.table('consignments').where('uam_owned',1).where('active',1).get()
    bands = db.table('consignment_band').get()
    
    for consignment in consignments:
        for invoice in invoices:
            #calculate fixed rate
            if consignment.consignment_code == invoice.consignment_code and consignment.fixed == 1:
                gp = consignment.current_rate * invoice.net
                #print(gp)
                daily_gp.append(gp)
            elif consignment.consignment_code == invoice.consignment_code and consignment.fixed == 0:
                total = consignment.current_net + invoice.net
                #print(total)
                for band in bands:
                    if consignment.id == band.consignment_id:
                        #calculate at band level1
                        if band.low == 0 and total < band.high:
                            gp1 = band.percent * invoice.net
                            #print(gp1)
                            daily_gp.append(gp1)
                        
                        elif band.low == 0 and consignment.current_net < band.high and total > band.high:
                            curPortion = band.high - consignment.current_net
                            gp2 = band.percent * curPortion
                            #print(gp2)
                            daily_gp.append(gp2)
                            nextPortion = invoice.net - curPortion
                            next_band = db.table('consignment_band').where('consignment_id', band.consignment_id).where('low', band.high + 1).first()
                            gp3 = nextPortion * next_band.percent
                            #print(gp3)
                            daily_gp.append(gp3)
                            break
                        
                          #calculate at band level2  
                        elif band.low != 0 and band.high != none and band.low <= total <= band.high:
                            gp4 = band.percent * invoice.net
                            #print(gp4)
                            daily_gp.append(gp4)
                            
                        elif band.low != 0 and band.high != none and band.low <= consignment.current_net <= band.high and total > band.high:
                            curPortion = band.high - consignment.current_net
                            gp5 = band.percent * curPortion
                            #print(gp5)
                            daily_gp.append(gp5)
                            nextPortion = invoice.net - curPortion
                            next_band = db.table('consignment_band').where('consignment_id', band.consignment_id).where('low', band.high + 1).first()
                            gp6 = nextPortion * next_band.percent
                            #print(gp6)
                            daily_gp.append(gp6)
                            #dict[consignment.consignment_code] = total
                            break    
                        
                        #calculate at band level2  
                        elif consignment.current_net >= band.low and band.high == none:
                            gp7 = band.percent * invoice.net
                            #print(gp7)
                            daily_gp.append(gp7)       
    
    return round(sum(daily_gp))
    
##DAILY MEMPHIS OWNED GROSS PROFIT PERCENTAGE

def dailyMemOwnedGrossProfitPercentage():
    if dailySales() == 0:
        GPPercent = 0
    else:
        GPPercent = round((memOwnedDailyGP()/dailySales()*100),2)
    return GPPercent

##MONTHLY MEMPHIS OWNED SALES

def monthlyMemOwnedSales():
    day_of_week = datetime.now().strftime("%w")
    year = str(datetime.now().year)
    month = str(datetime.now().month)
    today = str(date.today())
    weekday = today
    
    invoices = db.table('invoices').where('net', '>', 0).where('route_code', 'S').where('invoice_date', '<=', today).where('invoice_date', '>=', year + '-' + month + '-1').get() 
    consignments = db.table('consignments').where('uam_owned',1).where('active',1).get()
    
    total = []
    
    for consignment in consignments:
        for invoice in invoices:
            if consignment.consignment_code == invoice.consignment_code:
                total.append(invoice.net)
    sales = round(sum(total))
    return sales

##MONTHLY MEMPHIS OWNED GROSS PROFIT

def monthlyMemOwnedGP():
    day_of_week = datetime.now().strftime("%w")
    year = str(datetime.now().year)
    month = str(datetime.now().month)
    today = str(date.today())

    none = None
    daily_gp = []
    
    invoices = db.table('invoices').where('net', '>', 0).where('route_code', 'S').where('invoice_date','<=',today).where('invoice_date', '>=', year + '-' + month + '-1').get()
    consignments = db.table('consignments').where('uam_owned',1).where('active',1).get()
    bands = db.table('consignment_band').get()
    
    for consignment in consignments:
        for invoice in invoices:
            #calculate fixed rate
            if consignment.consignment_code == invoice.consignment_code and consignment.fixed == 1:
                gp = consignment.current_rate * invoice.net
                #print(gp)
                daily_gp.append(gp)
            elif consignment.consignment_code == invoice.consignment_code and consignment.fixed == 0:
                total = consignment.current_net + invoice.net
                #print(total)
                for band in bands:
                    if consignment.id == band.consignment_id:
                        #calculate at band level1
                        if band.low == 0 and total < band.high:
                            gp1 = band.percent * invoice.net
                            #print(gp1)
                            daily_gp.append(gp1)
                        
                        elif band.low == 0 and consignment.current_net < band.high and total > band.high:
                            curPortion = band.high - consignment.current_net
                            gp2 = band.percent * curPortion
                            #print(gp2)
                            daily_gp.append(gp2)
                            nextPortion = invoice.net - curPortion
                            next_band = db.table('consignment_band').where('consignment_id', band.consignment_id).where('low', band.high + 1).first()
                            gp3 = nextPortion * next_band.percent
                            #print(gp3)
                            daily_gp.append(gp3)
                        
                          #calculate at band level2  
                        elif band.low != 0 and band.high != none and band.low <= total <= band.high:
                            gp4 = band.percent * invoice.net
                            #print(gp4)
                            daily_gp.append(gp4)
                            
                        elif band.low != 0 and band.high != none and band.low <= consignment.current_net <= band.high and total > band.high:
                            curPortion = band.high - consignment.current_net
                            gp5 = band.percent * curPortion
                            #print(gp5)
                            daily_gp.append(gp5)
                            nextPortion = invoice.net - curPortion
                            next_band = db.table('consignment_band').where('consignment_id', band.consignment_id).where('low', band.high + 1).first()
                            gp6 = nextPortion * next_band.percent
                            #print(gp6)
                            daily_gp.append(gp6)
                            #dict[consignment.consignment_code] = total
                            break    
                            
                        #calculate at band level2  
                        elif consignment.current_net >= band.low and band.high == none:
                            gp7 = band.percent * invoice.net
                            #print(gp7)
                            daily_gp.append(gp7)       
    
    return round(sum(daily_gp))

##MONTHLY MEMPHIS OWNED GROSS PROFIT PERCENTAGE

def monthlyMemOwnedGPPercentage():
    GPPercent = round((monthlyMemOwnedGP()/monthlySales()*100),2)
    return GPPercent

##YEARLY MEMPHIS OWNED SALES

def yearlyMemOwnedSales():
    day_of_week = datetime.now().strftime("%w")
    year = str(datetime.now().year)
    month = str(datetime.now().month)
    today = str(date.today())
    weekday = today
    
    invoices = db.table('invoices').where('net', '>', 0).where('route_code', 'S').where('invoice_date', '<=', today).where('invoice_date', '>=', year + '-01-01').get() 
    consignments = db.table('consignments').where('uam_owned',1).where('active',1).get()
    
    total = []
    
    for consignment in consignments:
        for invoice in invoices:
            if consignment.consignment_code == invoice.consignment_code:
                total.append(invoice.net)
    sales = round(sum(total))
    return sales
##YEARLY MEMPHIS OWNED GROSS PROFIT

def yearlyMemOwnedGP():
    day_of_week = datetime.now().strftime("%w")
    year = str(datetime.now().year)
    month = str(datetime.now().month)
    today = str(date.today())

    none = None
    daily_gp = []
    
    invoices = db.table('invoices').where('net', '>', 0).where('route_code', 'S').where('invoice_date','<=',today).where('invoice_date', '>=', year + '-01-01').get()
    consignments = db.table('consignments').where('uam_owned',1).where('active',1).get()
    bands = db.table('consignment_band').get()
    
    for consignment in consignments:
        for invoice in invoices:
            #calculate fixed rate
            if consignment.consignment_code == invoice.consignment_code and consignment.fixed == 1:
                gp = consignment.current_rate * invoice.net
                #print(gp)
                daily_gp.append(gp)
            elif consignment.consignment_code == invoice.consignment_code and consignment.fixed == 0:
                total = consignment.current_net + invoice.net
                #print(total)
                for band in bands:
                    if consignment.id == band.consignment_id:
                        #calculate at band level1
                        if band.low == 0 and total < band.high:
                            gp1 = band.percent * invoice.net
                            #print(gp1)
                            daily_gp.append(gp1)
                        
                        elif band.low == 0 and consignment.current_net < band.high and total > band.high:
                            curPortion = band.high - consignment.current_net
                            gp2 = band.percent * curPortion
                            #print(gp2)
                            daily_gp.append(gp2)
                            nextPortion = invoice.net - curPortion
                            next_band = db.table('consignment_band').where('consignment_id', band.consignment_id).where('low', band.high + 1).first()
                            gp3 = nextPortion * next_band.percent
                            #print(gp3)
                            daily_gp.append(gp3)
                            break
                        
                          #calculate at band level2  
                        elif band.low != 0 and band.high != none and band.low <= total <= band.high:
                            gp4 = band.percent * invoice.net
                            #print(gp4)
                            daily_gp.append(gp4)
                            
                        elif band.low != 0 and band.high != none and band.low <= consignment.current_net <= band.high and total > band.high:
                            curPortion = band.high - consignment.current_net
                            gp5 = band.percent * curPortion
                            #print(gp5)
                            daily_gp.append(gp5)
                            nextPortion = invoice.net - curPortion
                            next_band = db.table('consignment_band').where('consignment_id', band.consignment_id).where('low', band.high + 1).first()
                            gp6 = nextPortion * next_band.percent
                            #print(gp6)
                            daily_gp.append(gp6)
                            #dict[consignment.consignment_code] = total
                            break    
                            
                        #calculate at band level2  
                        elif consignment.current_net >= band.low and band.high == none:
                            gp7 = band.percent * invoice.net
                            #print(gp7)
                            daily_gp.append(gp7)    
                            
    return round(sum(daily_gp))

##YEARLY MEMPHIS OWNED GROSS PROFIT PERCENTAGE

def yearlyMemOwnedGPPercentage():
    GPPercent = round((yearlyMemOwnedGP()/yearlySales()*100),2)
    return GPPercent

##DAILY CONSIGNED OWNED SALES

def dailyConsignedOwnedSales():
    day_of_week = datetime.now().strftime("%w")
    year = str(datetime.now().year)
    month = str(datetime.now().month)
    today = str(date.today())
    weekday = today
    
    invoices = db.table('invoices').where('net', '>', 0).where('route_code', 'S').where('invoice_date', today).get()
    consignments = db.table('consignments').where('uam_owned',0).where('active',1).get()
    
    total = []
    
    for consignment in consignments:
        for invoice in invoices:
            if consignment.consignment_code == invoice.consignment_code:
                total.append(invoice.net)
    sales = round(sum(total))
    return sales

##DAILY CONSIGNED OWNED GROSS PROFIT

def dailyConsignmentOwned():
    day_of_week = datetime.now().strftime("%w")
    year = str(datetime.now().year)
    month = str(datetime.now().month)
    today = str(date.today())

    none = None
    daily_gp = []
    
    invoices = db.table('invoices').where('net', '>', 0).where('route_code', 'S').where('invoice_date', today).get()
    consignments = db.table('consignments').where('uam_owned',0).where('active',1).get()
    bands = db.table('consignment_band').get()
    
    for consignment in consignments:
        for invoice in invoices:
            #calculate fixed rate
            if consignment.consignment_code == invoice.consignment_code and consignment.fixed == 1:
                gp = consignment.current_rate * invoice.net
                #print(gp)
                daily_gp.append(gp)
                
            elif consignment.consignment_code == invoice.consignment_code and consignment.fixed == 0:
                total = consignment.current_net + invoice.net
                #print(total)
                for band in bands:
                    if consignment.id == band.consignment_id:
                        
                        #calculate at band level1
                        if band.low == 0 and total < band.high:
                            gp1 = band.percent * invoice.net
                            #print(gp1)
                            daily_gp.append(gp1)
                        
                        elif band.low == 0 and consignment.current_net < band.high and total > band.high:
                            curPortion = band.high - consignment.current_net
                            gp2 = band.percent * curPortion
                            #print(gp2)
                            daily_gp.append(gp2)
                            nextPortion = invoice.net - curPortion
                            next_band = db.table('consignment_band').where('consignment_id', band.consignment_id).where('low', band.high + 1).first()
                            gp3 = nextPortion * next_band.percent
                            #print(gp3)
                            daily_gp.append(gp3)
                            break
                        
                          #calculate at band level2  
                        elif band.low != 0 and band.high != none and band.low <= total <= band.high:
                            gp4 = band.percent * invoice.net
                            #print(gp4)
                            daily_gp.append(gp4)
                            
                        elif band.low != 0 and band.high != none and band.low <= consignment.current_net <= band.high and total > band.high:
                            curPortion = band.high - consignment.current_net
                            gp5 = band.percent * curPortion
                            #print(gp5)
                            daily_gp.append(gp5)
                            nextPortion = invoice.net - curPortion
                            next_band = db.table('consignment_band').where('consignment_id', band.consignment_id).where('low', band.high + 1).first()
                            gp6 = nextPortion * next_band.percent
                            #print(gp6)
                            daily_gp.append(gp6)
                            #dict[consignment.consignment_code] = total
                            break    
                             
                        #calculate at band level2  
                        elif consignment.current_net >= band.low and band.high == none:
                            gp7 = band.percent * invoice.net
                            #print(gp7)
                            daily_gp.append(gp7)       
    
    return round(sum(daily_gp))
    
##DAILY CONSIGNED OWNED GROSS PROFIT PERCENTAGE

def dailyConsignedOwnedGPPercentage():
    if dailySales() == 0:
        GPPercent = 0
    else:
        GPPercent = round((dailyConsignmentOwned()/dailySales()*100),2)
    return GPPercent

##MONTHLY CONSIGNED OWNED SALES

def monthlyConsignedOwnedSales():
    day_of_week = datetime.now().strftime("%w")
    year = str(datetime.now().year)
    month = str(datetime.now().month)
    today = str(date.today())
    
    invoices = db.table('invoices').where('net', '>', 0).where('route_code', 'S').where('invoice_date','<=',today).where('invoice_date','>=',year + '-' + month + '-1').get()
    consignments = db.table('consignments').where('uam_owned',0).where('active',1).get()
    
    total = []
    
    for consignment in consignments:
        for invoice in invoices:
            if consignment.consignment_code == invoice.consignment_code:
                total.append(invoice.net)
    sales = round(sum(total))
    return sales
    
##MONTHLY CONSIGNED OWNED GROSS PROFIT

def monthlyConsignedOwned():
    day_of_week = datetime.now().strftime("%w")
    year = str(datetime.now().year)
    month = str(datetime.now().month)
    today = str(date.today())

    none = None
    daily_gp = []
    
    invoices = db.table('invoices').where('net', '>', 0).where('route_code', 'S').where('invoice_date','<=',today).where('invoice_date', '<=', today).where('invoice_date', '>=', year + '-' + month + '-1').get()
    consignments = db.table('consignments').where('uam_owned',0).where('active',1).get()
    bands = db.table('consignment_band').get()
    
    for consignment in consignments:
        for invoice in invoices:
            #calculate fixed rate
            if consignment.consignment_code == invoice.consignment_code and consignment.fixed == 1:
                gp = consignment.current_rate * invoice.net
                #print(gp)
                daily_gp.append(gp)
                
            elif consignment.consignment_code == invoice.consignment_code and consignment.fixed == 0:
                total = consignment.current_net + invoice.net
                #print(total)
                for band in bands:
                    if consignment.id == band.consignment_id:
                        
                        #calculate at band level1
                        if band.low == 0 and total < band.high:
                            gp1 = band.percent * invoice.net
                            #print(gp1)
                            daily_gp.append(gp1)
                            
                        elif band.low == 0 and consignment.current_net < band.high and total > band.high:
                            curPortion = band.high - consignment.current_net
                            gp2 = band.percent * curPortion
                            #print(gp2)
                            daily_gp.append(gp2)
                            nextPortion = invoice.net - curPortion
                            next_band = db.table('consignment_band').where('consignment_id', band.consignment_id).where('low', band.high + 1).first()
                            gp3 = nextPortion * next_band.percent
                            #print(gp3)
                            daily_gp.append(gp3)
                            break
                        
                          #calculate at band level2  
                        elif band.low != 0 and band.high != none and band.low <= total <= band.high:
                            gp4 = band.percent * invoice.net
                            #print(gp4)
                            daily_gp.append(gp4)
                            
                        elif band.low != 0 and band.high != none and band.low <= consignment.current_net <= band.high and total > band.high:
                            curPortion = band.high - consignment.current_net
                            gp5 = band.percent * curPortion
                            print(gp5)
                            daily_gp.append(gp5)
                            nextPortion = invoice.net - curPortion
                            next_band = db.table('consignment_band').where('consignment_id', band.consignment_id).where('low', band.high + 1).first()
                            gp6 = nextPortion * next_band.percent
                            print(gp6)
                            daily_gp.append(gp6)
                            #dict[consignment.consignment_code] = total
                            break  
                            
                        elif consignment.current_net >= band.low and band.high == none:
                            gp7 = band.percent * invoice.net
                            #print(gp7)
                            daily_gp.append(gp7)       
                        #calculate at band level2  
                    
    return round(sum(daily_gp))
    
##MONTHLY CONSIGNED OWNED GROSS PROFIT PERCENTAGE

def monthlyConsignedOwnedGPPercentage():
    GPPercent = round((monthlyConsignedOwned()/monthlySales()*100),2)
    return GPPercent

##YEARLY CONSIGNED OWNED SALES

def yearlyConsignedOwnedSales():    
    day_of_week = datetime.now().strftime("%w")
    year = str(datetime.now().year)
    month = str(datetime.now().month)
    today = str(date.today())
    weekday = today
    
    invoices = db.table('invoices').where('net', '>', 0).where('route_code', 'S').where('invoice_date','<=',today).where('invoice_date','>=',year + '-01-01').get()
    consignments = db.table('consignments').where('uam_owned',0).where('active',1).get()
    
    total = []
    
    for consignment in consignments:
        for invoice in invoices:
            if consignment.consignment_code == invoice.consignment_code:
                total.append(invoice.net)
    sales = round(sum(total))
    return sales

##YEARLY CONSIGNED OWNED GROSS PROFIT

def yearlyConsignedOwned():
    day_of_week = datetime.now().strftime("%w")
    year = str(datetime.now().year)
    month = str(datetime.now().month)
    today = str(date.today())

    none = None
    daily_gp = []
    
    invoices = db.table('invoices').where('net', '>', 0).where('route_code', 'S').where('invoice_date','<=',today).where('invoice_date', '>=', year + '-01-01').get()
    consignments = db.table('consignments').where('uam_owned',0).where('active',1).get()
    bands = db.table('consignment_band').get()
    
    for consignment in consignments:
        for invoice in invoices:
            #calculate fixed rate
            if consignment.consignment_code == invoice.consignment_code and consignment.fixed == 1:
                gp = consignment.current_rate * invoice.net
                #print(gp)
                daily_gp.append(gp)
            elif consignment.consignment_code == invoice.consignment_code and consignment.fixed == 0:
                total = consignment.current_net + invoice.net
                #print(total)
                for band in bands:
                    if consignment.id == band.consignment_id:
                        #calculate at band level1
                        if band.low == 0 and total < band.high:
                            gp1 = band.percent * invoice.net
                            #print(gp1)
                            daily_gp.append(gp1)
                        
                        elif band.low == 0 and consignment.current_net < band.high and total > band.high:
                            curPortion = band.high - consignment.current_net
                            gp2 = band.percent * curPortion
                            #print(gp2)
                            daily_gp.append(gp2)
                            nextPortion = invoice.net - curPortion
                            next_band = db.table('consignment_band').where('consignment_id', band.consignment_id).where('low', band.high + 1).first()
                            gp3 = nextPortion * next_band.percent
                            #print(gp3)
                            daily_gp.append(gp3)
                        
                          #calculate at band level2  
                        elif band.low != 0 and band.high != none and band.low <= total <= band.high:
                            gp4 = band.percent * invoice.net
                            #print(gp4)
                            daily_gp.append(gp4)
                            
                        elif band.low != 0 and band.high != none and band.low <= consignment.current_net <= band.high and total > band.high:
                            curPortion = band.high - consignment.current_net
                            gp5 = band.percent * curPortion
                            daily_gp.append(gp5)
                            nextPortion = invoice.net - curPortion
                            next_band = db.table('consignment_band').where('consignment_id', band.consignment_id).where('low', band.high + 1).first()
                            gp6 = nextPortion * next_band.percent
                            daily_gp.append(gp6)
                            #dict[consignment.consignment_code] = total
                            break    
                            
                        #calculate at band level2  
                        elif consignment.current_net >= band.low and band.high == none:
                            gp7 = band.percent * invoice.net
                            #print(gp7)
                            daily_gp.append(gp7)    
                            
    return round(sum(daily_gp))
    
##YEARLY CONSIGNED OWNED GROSS PROFIT PERCENTAGE

def yearlyConsignedOwnedGPPercentage():
    GPPercent = round((yearlyConsignedOwned()/yearlySales()*100),2)
    return GPPercent

##DAILY MEMPHIS LOCATION SALES

def dailyMemLocationSales():
    year = str(datetime.now().year)
    month = str(datetime.now().month)
    today = str(date.today())

    weekday = today

    if day_of_week in ['1', '2', '3', '4', '5']:
        if db.table('invoices').where_not_in('salesperson', ['YJORDAN2', 'DMULLER', 'CCREASEY']).where('route_code', 'S').where('net','>','0').where('invoice_date', '=', today).sum('net') is not None:
            daily_totals = round(db.table('invoices').where_not_in('salesperson', ['YJORDAN2', 'DMULLER', 'CCREASEY']).where('route_code', 'S').where('net', '>', '0').where('invoice_date', '=', today).sum('net'))
            return daily_totals
        else:
            daily_totals = 0
            return daily_totals
    elif day_of_week == '6':
        daily_totals = round(db.table('invoices').where_not_in('salesperson', ['YJORDAN2', 'DMULLER', 'CCREASEY']).where('route_code', 'S').where('net', '>', '0').where('invoice_date', '=', (datetime.now() - timedelta(1)).strftime('%Y-%m-%d')).sum('net'))
        return daily_totals
    elif day_of_week == '0':
        daily_totals = round(db.table('invoices').where_not_in('salesperson', ['YJORDAN2', 'DMULLER', 'CCREASEY']).where('route_code', 'S').where('net', '>', 0).where('invoice_date', '=', (datetime.now() - timedelta(2)).strftime('%Y-%m-%d')).sum('net'))
        return daily_totals

##DAILY MEMPHIS LOCATION GROSS PROFIT

def dailyMemLocationGP():
    day_of_week = datetime.now().strftime("%w")
    year = str(datetime.now().year)
    month = str(datetime.now().month)
    today = str(date.today())

    none = None
    daily_gp = []

    invoices = db.table('invoices').where('net', '>', 0).where('route_code', 'S').where_not_in('salesperson', ['YJORDAN2', 'DMULLER', 'CCREASEY']).where('invoice_date','=', today).get()
    consignments = db.table('consignments').get()
    bands = db.table('consignment_band').get()
    
    for consignment in consignments:
        for invoice in invoices:
            #calculate fixed rate
            if consignment.consignment_code == invoice.consignment_code and consignment.fixed == 1:
                gp = consignment.current_rate * invoice.net
                #print(gp)
                daily_gp.append(gp)
            elif consignment.consignment_code == invoice.consignment_code and consignment.fixed == 0:
                total = consignment.current_net + invoice.net
                #print(total)
                for band in bands:
                    if consignment.id == band.consignment_id:
                        #calculate at band level1
                        if band.low == 0 and total < band.high:
                            gp1 = band.percent * invoice.net
                            #print(gp1)
                            daily_gp.append(gp1)
                        
                        elif band.low == 0 and consignment.current_net < band.high and total > band.high:
                            curPortion = band.high - consignment.current_net
                            gp2 = band.percent * curPortion
                            #print(gp2)
                            daily_gp.append(gp2)
                            nextPortion = invoice.net - curPortion
                            next_band = db.table('consignment_band').where('consignment_id', band.consignment_id).where('low', band.high + 1).first()
                            gp3 = nextPortion * next_band.percent
                            #print(gp3)
                            daily_gp.append(gp3)
                            break
                        
                          #calculate at band level2  
                        elif band.low != 0 and band.high != none and band.low <= total <= band.high:
                            gp4 = band.percent * invoice.net
                            #print(gp4)
                            daily_gp.append(gp4)
                            
                        elif band.low != 0 and band.high != none and band.low <= consignment.current_net <= band.high and total > band.high:
                            curPortion = band.high - consignment.current_net
                            gp5 = band.percent * curPortion
                            #print(gp5)
                            daily_gp.append(gp5)
                            nextPortion = invoice.net - curPortion
                            next_band = db.table('consignment_band').where('consignment_id', band.consignment_id).where('low', band.high + 1).first()
                            gp6 = nextPortion * next_band.percent
                            #print(gp6)
                            daily_gp.append(gp6)
                            #dict[consignment.consignment_code] = total
                            break    
                            
                        #calculate at band level2  
                        elif consignment.current_net >= band.low and band.high == none:
                            gp7 = band.percent * invoice.net
                            #print(gp7)
                            daily_gp.append(gp7)       
    
    return round(sum(daily_gp))

##DAILY MEMPHIS LOCATION GROSS PROFIT PERCENTAGE

def dailymemLocationGPPercentage():
    if dailySales() == 0:
        GPPercent = 0
    else:
        GPPercent = round((dailyMemLocationGP()/dailySales()*100),2)
    return GPPercent

##MONTHLY MEMPHIS LOCATION SALES

def monthlyMemLocationSales():
    year = str(datetime.now().year)
    month = str(datetime.now().month)
    today = str(date.today())

    total_mem_location_monthly_sales = round(db.table('invoices').where_not_in('salesperson', ['YJORDAN2','DMULLER','CCREASEY']).where('net', '>', '0').where('invoice_date', '<', today).where('invoice_date', '>', year + '-' + month + '-01').sum('net'))
    #total_mem_location_monthly_sales = round(db.table('invoices').where_not_in('salesperson', ['YJORDAN2','DMULLER','CCREASEY']).where('route_code', 'S').where('net', '>', '0').where('invoice_date', '<=', '2020-03-24').where('invoice_date', '>=', '2020-03-01').sum('net'))
    return total_mem_location_monthly_sales

##MONTHLY MEMPHIS LOCATION GROSS PROFIT

def monthlyMemLocationGP():
    day_of_week = datetime.now().strftime("%w")
    year = str(datetime.now().year)
    month = str(datetime.now().month)
    today = str(date.today())

    none = None
    daily_gp = []

    invoices = db.table('invoices').where('net', '>', 0).where('route_code', 'S').where_not_in('salesperson', ['YJORDAN2', 'DMULLER', 'CCREASEY']).where('invoice_date', '<=', today).where('invoice_date', '>=', year + '-' + month + '-1').get()
    consignments = db.table('consignments').get()
    bands = db.table('consignment_band').get()
    
    for consignment in consignments:
        for invoice in invoices:
            #calculate fixed rate
            if consignment.consignment_code == invoice.consignment_code and consignment.fixed == 1:
                gp = consignment.current_rate * invoice.net
                #print(gp)
                daily_gp.append(gp)
            elif consignment.consignment_code == invoice.consignment_code and consignment.fixed == 0:
                total = consignment.current_net + invoice.net
                #print(total)
                for band in bands:
                    if consignment.id == band.consignment_id:
                        #calculate at band level1
                        if band.low == 0 and total < band.high:
                            gp1 = band.percent * invoice.net
                            #print(gp1)
                            daily_gp.append(gp1)
                        
                        elif band.low == 0 and consignment.current_net < band.high and total > band.high:
                            curPortion = band.high - consignment.current_net
                            gp2 = band.percent * curPortion
                            #print(gp2)
                            daily_gp.append(gp2)
                            nextPortion = invoice.net - curPortion
                            next_band = db.table('consignment_band').where('consignment_id', band.consignment_id).where('low', band.high + 1).first()
                            gp3 = nextPortion * next_band.percent
                            #print(gp3)
                            daily_gp.append(gp3)
                            break
                        
                          #calculate at band level2  
                        elif band.low != 0 and band.high != none and band.low <= total <= band.high:
                            gp4 = band.percent * invoice.net
                            #print(gp4)
                            daily_gp.append(gp4)
                            
                        elif band.low != 0 and band.high != none and band.low <= consignment.current_net <= band.high and total > band.high:
                            curPortion = band.high - consignment.current_net
                            gp5 = band.percent * curPortion
                            #print(gp5)
                            daily_gp.append(gp5)
                            nextPortion = invoice.net - curPortion
                            next_band = db.table('consignment_band').where('consignment_id', band.consignment_id).where('low', band.high + 1).first()
                            gp6 = nextPortion * next_band.percent
                            #print(gp6)
                            daily_gp.append(gp6)
                            #dict[consignment.consignment_code] = total
                            break    
                            
                        #calculate at band level2  
                        elif consignment.current_net >= band.low and band.high == none:
                            gp7 = band.percent * invoice.net
                            #print(gp7)
                            daily_gp.append(gp7)       
    
    return round(sum(daily_gp))
    
##MONTHLY MEMPHIS LOCATION GROSS PROFIT PERCENTAGE

def monthlyMemLocationGPPercentage():
    GPPercent = round((monthlyMemLocationGP()/monthlySales()*100),2)
    return GPPercent

##YEARLY MEMPHIS LOCATION SALES

def yearlyMemLocationSales():
    yearly_totals = round(db.table('invoices').where_not_in('salesperson', ['YJORDAN2', 'DMULLER', 'CCREASEY']).where('net', '>', '0').where('invoice_date', '<=', today).where('invoice_date', '>=', year + '-' + '01-01').sum('net'))
    #yearly_totals = round(invoice.where_not_in('salesperson', ['YJORDAN2', 'DMULLER', 'CCREASEY']).where('route_code', 'S').where('net', '>', '0').where('invoice_date', '<=', '2020-02-03').where('invoice_date', '>=', '2020-01-01').sum('net'))
    return yearly_totals

##YEARLY MEMPHIS LOCATION GROSS PROFIT

def yearlyMemLocationGP():
    day_of_week = datetime.now().strftime("%w")
    year = str(datetime.now().year)
    month = str(datetime.now().month)
    today = str(date.today())

    none = None
    daily_gp = []

    invoices = db.table('invoices').where('net', '>', 0).where('route_code', 'S').where_not_in('salesperson', ['YJORDAN2', 'DMULLER', 'CCREASEY']).where('invoice_date', '<=', today).where('invoice_date', '>=', year  + '-01-01').get()
    consignments = db.table('consignments').get()
    bands = db.table('consignment_band').get()
    
    for consignment in consignments:
        for invoice in invoices:
            #calculate fixed rate
            if consignment.consignment_code == invoice.consignment_code and consignment.fixed == 1:
                gp = consignment.current_rate * invoice.net
                #print(gp)
                daily_gp.append(gp)
            elif consignment.consignment_code == invoice.consignment_code and consignment.fixed == 0:
                total = consignment.current_net + invoice.net
                #print(total)
                for band in bands:
                    if consignment.id == band.consignment_id:
                        #calculate at band level1
                        if band.low == 0 and total < band.high:
                            gp1 = band.percent * invoice.net
                            #print(gp1)
                            daily_gp.append(gp1)
                        
                        elif band.low == 0 and consignment.current_net < band.high and total > band.high:
                            curPortion = band.high - consignment.current_net
                            gp2 = band.percent * curPortion
                            #print(gp2)
                            daily_gp.append(gp2)
                            nextPortion = invoice.net - curPortion
                            next_band = db.table('consignment_band').where('consignment_id', band.consignment_id).where('low', band.high + 1).first()
                            gp3 = nextPortion * next_band.percent
                            #print(gp3)
                            daily_gp.append(gp3)
                        
                          #calculate at band level2  
                        elif band.low != 0 and band.high != none and band.low <= total <= band.high:
                            gp4 = band.percent * invoice.net
                            #print(gp4)
                            daily_gp.append(gp4)
                            
                        elif band.low != 0 and band.high != none and band.low <= consignment.current_net <= band.high and total > band.high:
                            curPortion = band.high - consignment.current_net
                            gp5 = band.percent * curPortion
                            #print(gp5)
                            daily_gp.append(gp5)
                            nextPortion = invoice.net - curPortion
                            next_band = db.table('consignment_band').where('consignment_id', band.consignment_id).where('low', band.high + 1).first()
                            gp6 = nextPortion * next_band.percent
                            #print(gp6)
                            daily_gp.append(gp6)
                            #dict[consignment.consignment_code] = total
                            break    
                            
                        #calculate at band level2  
                        elif consignment.current_net >= band.low and band.high == none:
                            gp7 = band.percent * invoice.net
                            #print(gp7)
                            daily_gp.append(gp7)       
    
    return round(sum(daily_gp))
    
##YEARHLY MEMPHIS LOCATION GROSS PROFIT PERCENTAGE

def yearlyMemLocationGPPercentage():
    GPPercent = round((yearlyMemLocationGP()/yearlySales()*100),2)
    return GPPercent

##DAILY UK LOCATION SALES

def dailyUKLocationSales():
    day_of_week = datetime.now().strftime("%w")
    year = str(datetime.now().year)
    month = str(datetime.now().month)
    today = str(date.today())
    
    if day_of_week in ['1', '2', '3', '4', '5']:
        if db.table('invoices').where_in('salesperson', ['YJORDAN2', 'DMULLER', 'CCREASEY']).where('route_code', 'S').where('net', '>', '0').where('invoice_date', '=', today).sum('net') is not None:
            daily_totals = round(db.table('invoices').where_in('salesperson', ['YJORDAN2', 'DMULLER', 'CCREASEY']).where('route_code','S').where('net','>','0').where('invoice_date', '=', today).sum('net'))
            return daily_totals
        else:
            daily_totals = 0
            return daily_totals
    elif day_of_week == '6':
        daily_totals = round(db.table('invoices').where_in('salesperson', ['YJORDAN2', 'DMULLER','CCREASEY']).where('route_code', 'S').where('net', '>', 0).where('invoice_date', '=', (datetime.now() - timedelta(1)).strftime('%Y-%m-%d')).sum('net'))
        return daily_totals
    elif day_of_week == '0':
        daily_totals = round(db.table('invoices').where_in('salesperson', ['YJORDAN2', 'DMULLER', 'CCREASEY']).where('route_code', 'S').where('net', '>', 0).where('invoice_date', '=', (datetime.now() - timedelta(2)).strftime('%Y-%m-%d')).sum('net'))
        return daily_totals

##DAILY UK LOCATION GROSS PROFIT

def dailyUKLocationGP():
    day_of_week = datetime.now().strftime("%w")
    year = str(datetime.now().year)
    month = str(datetime.now().month)
    today = str(date.today())

    none = None
    daily_gp = []

    invoices = db.table('invoices').where('net', '>', 0).where('route_code', 'S').where_in('salesperson', ['YJORDAN2', 'DMULLER', 'CCREASEY']).where('invoice_date', today).get()
    consignments = db.table('consignments').get()
    bands = db.table('consignment_band').get()
    
    for consignment in consignments:
        for invoice in invoices:
            #calculate fixed rate
            if consignment.consignment_code == invoice.consignment_code and consignment.fixed == 1:
                gp = consignment.current_rate * invoice.net
                #print(gp)
                daily_gp.append(gp)
            elif consignment.consignment_code == invoice.consignment_code and consignment.fixed == 0:
                total = consignment.current_net + invoice.net
                #print(total)
                for band in bands:
                    if consignment.id == band.consignment_id:
                        #calculate at band level1
                        if band.low == 0 and total < band.high:
                            gp1 = band.percent * invoice.net
                            #print(gp1)
                            daily_gp.append(gp1)
                        
                        elif band.low == 0 and consignment.current_net < band.high and total > band.high:
                            curPortion = band.high - consignment.current_net
                            gp2 = band.percent * curPortion
                            #print(gp2)
                            daily_gp.append(gp2)
                            nextPortion = invoice.net - curPortion
                            next_band = db.table('consignment_band').where('consignment_id', band.consignment_id).where('low', band.high + 1).first()
                            gp3 = nextPortion * next_band.percent
                            #print(gp3)
                            daily_gp.append(gp3)
                            break
                        
                          #calculate at band level2  
                        elif band.low != 0 and band.high != none and band.low <= total <= band.high:
                            gp4 = band.percent * invoice.net
                            #print(gp4)
                            daily_gp.append(gp4)
                            
                        elif band.low != 0 and band.high != none and band.low <= consignment.current_net <= band.high and total > band.high:
                            curPortion = band.high - consignment.current_net
                            gp5 = band.percent * curPortion
                            #print(gp5)
                            daily_gp.append(gp5)
                            nextPortion = invoice.net - curPortion
                            next_band = db.table('consignment_band').where('consignment_id', band.consignment_id).where('low', band.high + 1).first()
                            gp6 = nextPortion * next_band.percent
                            #print(gp6)
                            daily_gp.append(gp6)
                            #dict[consignment.consignment_code] = total
                            break    
                            
                        #calculate at band level2  
                        elif consignment.current_net >= band.low and band.high == none:
                            gp7 = band.percent * invoice.net
                            #print(gp7)
                            daily_gp.append(gp7)       
    
    return round(sum(daily_gp))
    
    
##DAILY UK LOCATION GROSS PROFIT PERCENTAGE

def dailyUKLocationGPPercentage():
    if dailySales() == 0:
        GPPercent = 0
    else:
        GPPercent = round((dailyUKLocationGP()/dailySales()*100),2)
    return GPPercent

##MONTHLY UK LOCATION SALES

def monthlyUKLocationSales():
    total_uk_location_monthly_sales = round(db.table('invoices').where_in('salesperson', ['YJORDAN2', 'DMULLER', 'CCREASEY']).where('net', '>', 0).where('invoice_date', '<=', today).where('invoice_date', '>=', year + '-' + month + '-1').sum('net'))
    #total_uk_location_monthly_sales = round(invoice.where_in('salesperson', ['YJORDAN2', 'DMULLER', 'CCREASEY']).where('route_code', 'S').where('net', '>', 0).where('invoice_date', '<=', '2020-02-03').where('invoice_date', '>=', '2020-02-01').sum('net'))
    return total_uk_location_monthly_sales

##MONTHLY UK LOCATION GROSS PROFIT

def monthlyUKLocationGP():
    day_of_week = datetime.now().strftime("%w")
    year = str(datetime.now().year)
    month = str(datetime.now().month)
    today = str(date.today())

    none = None
    daily_gp = []

    invoices = db.table('invoices').where('net', '>', 0).where('route_code', 'S').where_in('salesperson', ['YJORDAN2', 'DMULLER', 'CCREASEY']).where('invoice_date','<=',today).where('invoice_date', '>=', year + '-' + month + '-1').get()
    consignments = db.table('consignments').get()
    bands = db.table('consignment_band').get()
    
    for consignment in consignments:
        for invoice in invoices:
            #calculate fixed rate
            if consignment.consignment_code == invoice.consignment_code and consignment.fixed == 1:
                gp = consignment.current_rate * invoice.net
                #print(gp)
                daily_gp.append(gp)
            elif consignment.consignment_code == invoice.consignment_code and consignment.fixed == 0:
                total = consignment.current_net + invoice.net
                #print(total)
                for band in bands:
                    if consignment.id == band.consignment_id:
                        #calculate at band level1
                        if band.low == 0 and total < band.high:
                            gp1 = band.percent * invoice.net
                            #print(gp1)
                            daily_gp.append(gp1)
                        
                        elif band.low == 0 and consignment.current_net < band.high and total > band.high:
                            curPortion = band.high - consignment.current_net
                            gp2 = band.percent * curPortion
                            #print(gp2)
                            daily_gp.append(gp2)
                            nextPortion = invoice.net - curPortion
                            next_band = db.table('consignment_band').where('consignment_id', band.consignment_id).where('low', band.high + 1).first()
                            gp3 = nextPortion * next_band.percent
                            #print(gp3)
                            daily_gp.append(gp3)
                            break
                        
                          #calculate at band level2  
                        elif band.low != 0 and band.high != none and band.low <= total <= band.high:
                            gp4 = band.percent * invoice.net
                            #print(gp4)
                            daily_gp.append(gp4)
                            
                        elif band.low != 0 and band.high != none and band.low <= consignment.current_net <= band.high and total > band.high:
                            curPortion = band.high - consignment.current_net
                            gp5 = band.percent * curPortion
                            #print(gp5)
                            daily_gp.append(gp5)
                            nextPortion = invoice.net - curPortion
                            next_band = db.table('consignment_band').where('consignment_id', band.consignment_id).where('low', band.high + 1).first()
                            gp6 = nextPortion * next_band.percent
                            #print(gp6)
                            daily_gp.append(gp6)
                            #dict[consignment.consignment_code] = total
                            break    
                            
                        #calculate at band level2  
                        elif consignment.current_net >= band.low and band.high == none:
                            gp7 = band.percent * invoice.net
                            #print(gp7)
                            daily_gp.append(gp7)       
    
    return round(sum(daily_gp))

##MONTHLY UK LOCATION GROSS PROFIT PERCENTAGE

def monthlyUKLocationGPPercentage():
    GPPercent = round((monthlyUKLocationGP()/monthlySales()*100),2)
    return GPPercent

##YEARLY UK LOCATION SALES

def yearlyUKLocationSales():
    yearly_totals = round(db.table('invoices').where_in('salesperson', ['YJORDAN2', 'DMULLER','CCREASEY']).where('net', '>', 0).where('invoice_date', '<', today).where('invoice_date', '>', year + '-' + '01-01').sum('net'))
    #yearly_totals = round(invoice.where_in('salesperson', ['YJORDAN2', 'DMULLER','CCREASEY']).where('net', '>', 0).where('route_code', 'S').where('invoice_date', '<=', '2020-02-03').where('invoice_date', '>=', '2020-01-01').sum('net'))
    return yearly_totals

##YEARL UK LOCATION GROSS PROFIT

def yearlyUKLocationGP():
    day_of_week = datetime.now().strftime("%w")
    year = str(datetime.now().year)
    month = str(datetime.now().month)
    today = str(date.today())

    none = None
    daily_gp = []

    invoices = db.table('invoices').where('net', '>', 0).where('route_code', 'S').where_in('salesperson', ['YJORDAN2', 'DMULLER', 'CCREASEY']).where('invoice_date', '<=', today).where('invoice_date', '>=', year + '-' + '01-01').get()
    consignments = db.table('consignments').get()
    bands = db.table('consignment_band').get()
    
    for consignment in consignments:
        for invoice in invoices:
            #calculate fixed rate
            if consignment.consignment_code == invoice.consignment_code and consignment.fixed == 1:
                gp = consignment.current_rate * invoice.net
                #print(gp)
                daily_gp.append(gp)
                
            elif consignment.consignment_code == invoice.consignment_code and consignment.fixed == 0:
                total = consignment.current_net + invoice.net
                #print(total)
                for band in bands:
                    if consignment.id == band.consignment_id:
                        
                        #calculate at band level1
                        if band.low == 0 and total < band.high:
                            gp1 = band.percent * invoice.net
                            #print(gp1)
                            daily_gp.append(gp1)
                        
                        elif band.low == 0 and consignment.current_net < band.high and total > band.high:
                            curPortion = band.high - consignment.current_net
                            gp2 = band.percent * curPortion
                            #print(gp2)
                            daily_gp.append(gp2)
                            nextPortion = invoice.net - curPortion
                            next_band = db.table('consignment_band').where('consignment_id', band.consignment_id).where('low', band.high + 1).first()
                            gp3 = nextPortion * next_band.percent
                            #print(gp3)
                            daily_gp.append(gp3)
                            break
                        
                          #calculate at band level2  
                        elif band.low != 0 and band.high != none and band.low <= total <= band.high:
                            gp4 = band.percent * invoice.net
                            #print(gp4)
                            daily_gp.append(gp4)
                            
                        elif band.low != 0 and band.high != none and band.low <= consignment.current_net <= band.high and total > band.high:
                            curPortion = band.high - consignment.current_net
                            gp5 = band.percent * curPortion
                            #print(gp5)
                            daily_gp.append(gp5)
                            nextPortion = invoice.net - curPortion
                            next_band = db.table('consignment_band').where('consignment_id', band.consignment_id).where('low', band.high + 1).first()
                            gp6 = nextPortion * next_band.percent
                            #print(gp6)
                            daily_gp.append(gp6)
                            #dict[consignment.consignment_code] = total
                            break    
                            
                        #calculate at band level2  
                        elif consignment.current_net >= band.low and band.high == none:
                            gp7 = band.percent * invoice.net
                            #print(gp7)
                            daily_gp.append(gp7)       
    
    return round(sum(daily_gp))
    
##YEARLY UK LOCATION GROSS PROFIT PERCENTAGE

def yearlyUKLocationGPPercentage():
    GPPercent = round((yearlyUKLocationGP()/yearlySales()*100),2)
    return GPPercent

##DAILY UNKNOWN CONSIGNMENT OWNED SALES

def dailyUnknownConsignedSales():
    unknown = dailySales() - (dailyMemOwnedSales() + dailyConsignedOwnedSales())
    return unknown

##DAILY UNKNOWN LOCATION SALES

def dailyUnknownLocationSales():
    unknown = dailySales() - (dailyMemLocationSales() + dailyUKLocationSales())
    return unknown

##MONTHLY UNKNOWN CONSIGNMENT OWNED SALES

def monthlyUnknownConsignedSales():
    unknown = monthlySales() - (monthlyMemOwnedSales() + monthlyConsignedOwnedSales())
    return unknown

##MONTHLY UNKNOWN LOCATION SALES

def monthlyUnknownLocationSales():
    unknown = monthlySales() - (monthlyMemLocationSales() + monthlyUKLocationSales())
    return unknown

##DAILY UNKNOWN CONSIGNMENT GROSS PROFIT

def dailyUnknownConsignmentGrossProfit():
    unknownGP = dailyGrossProft() - (memOwnedDailyGP() + dailyConsignmentOwned())
    return unknownGP

##DAILY UNKNOWN CONSIGNMENT GROSS PROFIT PERCENTAGE

def dailyUnknownConsignmentGPPercentage():
    unknownGPPercentage = dailyGrossProfitPercentage() - (dailyMemOwnedGrossProfitPercentage() + dailyConsignedOwnedGPPercentage())
    return unknownGPPercentage

##DAILY UNKNOWN LOCATION GROSS PROFIT

def dailyUnknownLocationGrossProfit():
    unknownGP = dailyGrossProft() - (dailyMemLocationGP() + dailyUKLocationGP())
    return unknownGP

##DAILY UNKNOWN LOCATION GROSS PROFIT PERCENTAGE

def dailyUnknownLocationGPPercentage():
    unknownGPPercentage = dailyGrossProfitPercentage() - (dailymemLocationGPPercentage() + dailyUKLocationGPPercentage())
    return unknownGPPercentage

##MONTHLY UNKNOWN CONSIGNMENT GROSS PROFIT

def monthlyUnknownConsignmentGrossProfit():
    unknownGP = monthlyGrossProfit() - (monthlyMemOwnedGP() + monthlyConsignedOwned())
    return unknownGP

##MONTHLY UNKNOWN CONSIGNMENT GROSS PROFIT PERCENTAGE

def monthlyUnknownConsignmentGPPercentage():
    unknownGPPercentage = monthlyGrossProfitPercentage() - (monthlyMemOwnedGPPercentage() + monthlyConsignedOwnedGPPercentage())
    return unknownGPPercentage

##MONTHLY UNKNOWN LOCATION GROSS PROFIT

def monthlyUnknownLocationGrossProfit():
    unknownGP = monthlyGrossProfit() - (monthlyMemLocationGP() + monthlyUKLocationGP())
    return unknownGP

##MONTHLY UNKNOWN LOCATION GROSS PROFIT PERCENTAGE

def monthlyUnknownLocationGPPercentage():
    unknownGPPercentage = monthlyGrossProfitPercentage() - (monthlyMemLocationGPPercentage() + monthlyUKLocationGPPercentage())
    return unknownGPPercentage

##YEARLY UNKNOWN CONSIGNMENT GROSS PROFIT

def yearlyUnknownConsignmentGrossProfit():
    unknownGP = yearlyGrossProfit() - (yearlyMemOwnedGP() + yearlyConsignedOwned())
    return unknownGP

##YEARLY UNKNOWN CONSIGNMENT GROSS PROFIT PERCENTAGE

def yearlyUnknownConsignmentGPPercentage():
    unknownGPPercentage = yearlyGrossProfitPercentage() - (yearlyMemOwnedGPPercentage() + yearlyConsignedOwnedGPPercentage())
    return unknownGPPercentage

##YEARLY UNKOWN LOCATION GROSS PROFIT

def yearlyUnknownLocationGrossProft():
    unknownGP = yearlyGrossProfit() - (yearlyMemLocationGP() + yearlyUKLocationGP())
    return unknownGP

##YEARLY UNKNOWN LOCATION GROSS PROFIT PERCENTAGE

def yearlyUnknownLocatoinGPPercentage():
    unknownGPPercentage = yearlyGrossProfitPercentage() - (yearlyMemLocationGPPercentage() + yearlyUKLocationGPPercentage())
    return unknownGPPercentage

## INSERTS DATA INTO DATABASE

def GrossProfitReport():
    import MySQLdb
    import mysql.connector
    
    mydb = mysql.connector.connect(host = 'localhost', user = 'root', password='password', database='atc')
    mycursor = mydb.cursor()
    
    functions = [memOwnedDailyGP(),dailyConsignmentOwned(),dailyUnknownConsignmentGrossProfit(),dailyMemLocationGP(),dailyUKLocationGP(),
            dailyUnknownLocationGrossProfit(),monthlyMemOwnedGP(),monthlyConsignedOwned(),monthlyUnknownConsignmentGrossProfit(),
            monthlyMemLocationGP(),monthlyUKLocationGP(),monthlyUnknownLocationGrossProfit(),yearlyMemOwnedGP(),yearlyConsignedOwned(),
            yearlyUnknownConsignmentGrossProfit(),yearlyMemLocationGP(),yearlyUKLocationGP(),yearlyUnknownLocationGrossProft(),
            dailyMemOwnedGrossProfitPercentage(),dailyConsignedOwnedGPPercentage(),dailyUnknownConsignmentGPPercentage(),dailymemLocationGPPercentage(),
            dailyUKLocationGPPercentage(),dailyUnknownLocationGPPercentage(),monthlyMemOwnedGPPercentage(),monthlyConsignedOwnedGPPercentage(),
            monthlyUnknownConsignmentGPPercentage(),monthlyMemLocationGPPercentage(),monthlyUKLocationGPPercentage(),monthlyUnknownLocationGPPercentage(),
            yearlyMemOwnedGPPercentage(),yearlyConsignedOwnedGPPercentage(),yearlyUnknownConsignmentGPPercentage(),yearlyMemLocationGPPercentage(),
            yearlyUKLocationGPPercentage(),yearlyUnknownLocatoinGPPercentage(),dailyMemOwnedSales(),dailyConsignedOwnedSales(),dailyUnknownConsignedSales(),
            dailyMemLocationSales(),dailyUKLocationSales(),dailyUnknownLocationSales(),monthlyMemOwnedSales(),monthlyConsignedOwnedSales(),monthlyUnknownConsignedSales(),
            monthlyMemLocationSales(),monthlyUKLocationSales(),monthlyUnknownLocationSales()]
    
    insert_stmt = ('insert into report_gross_profits (day_owned, day_consigned, day_unknown, day_mem, day_uk, day_other, mtd_owned, mtd_consigned, mtd_unknown, mtd_mem, mtd_uk, mtd_other, ytd_owned, ytd_consigned, ytd_unknown, ytd_mem, ytd_uk, ytd_other, percent_day_owned, percent_day_consigned, percent_day_unknown, percent_day_mem, percent_day_uk, percent_day_other, percent_mtd_owned, percent_mtd_consigned, percent_mtd_unknown, percent_mtd_mem, percent_mtd_uk, percent_mtd_other, percent_ytd_owned, percent_ytd_consigned, percent_ytd_unknown, percent_ytd_mem, percent_ytd_uk, percent_ytd_other, total_day_owned, total_day_consigned, total_day_unknown, total_day_mem, total_day_uk, total_day_other, total_mtd_owned, total_mtd_consigned, total_mtd_unknown, total_mtd_mem, total_mtd_uk, total_mtd_other) values (%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s)')
    data = functions
    
    mycursor.execute(insert_stmt, data)
    mydb.commit()
    
GrossProfitReport()
    